In [1]:
import pandas as pd, numpy as np
from sklearn.model_selection import KFold, GroupKFold
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import f1_score

In [2]:
train = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train.csv')

(13174211, 20)


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


In [3]:
targets = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]) )
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

(212022, 4)


,session_id,correct,session,q
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090314121766812_q1,1,20090314121766812,1
3,20090314363702160_q1,1,20090314363702160,1
4,20090314441803444_q1,1,20090314441803444,1


In [4]:
train['room_coor_x'] = train['room_coor_x'].fillna(0)
train['room_coor_y'] = train['room_coor_y'].fillna(0)
train['screen_coor_x'] = train['screen_coor_x'].fillna(0)
train['screen_coor_y'] = train['screen_coor_y'].fillna(0)
train['hover_duration'] = train['hover_duration'].fillna(0)
train['page'] = train['page'].fillna(-1)

In [5]:
targets = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]) )
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
print( targets.shape )
targets.head()

(212022, 4)


,session_id,correct,session,q
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090314121766812_q1,1,20090314121766812,1
3,20090314363702160_q1,1,20090314363702160,1
4,20090314441803444_q1,1,20090314441803444,1


In [6]:
NUNIQUE = ['fqid', 'room_fqid', 'text','event_name']

COUNT = ['text','fqid']

MMS = ['elapsed_time', 'hover_duration','index'] 
MEAN = ['level','page']
STD = ['room_coor_x', 'room_coor_y','screen_coor_x','screen_coor_y']
EVENTS = ['navigate_click','person_click','cutscene_click','object_click','notification_click',
          'map_click','observation_click','checkpoint','notebook_click']

EVENTS = ['navigate_click','person_click','cutscene_click','object_click',
          'map_hover','notification_click','map_click','observation_click',
          'checkpoint','notebook_click']


In [7]:
def feature_engineer(train):
    dfs = []
    for c in NUNIQUE:
        tmp = train.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in COUNT:
        tmp = train.groupby(['session_id','level_group'])[c].agg('count')
        tmp.name = tmp.name + '_count'
        dfs.append(tmp)

    for c in MMS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('max')
        tmp.name = tmp.name + '_max'
        dfs.append(tmp)
    for c in MMS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('min')
        tmp.name = tmp.name + '_min'
        dfs.append(tmp)
    for c in MMS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('mean')
        tmp.name = tmp.name + '_mean'
        dfs.append(tmp)
    for c in MEAN:
        tmp = train.groupby(['session_id','level_group'])[c].agg('mean')
        tmp.name = tmp.name + '_mean'
        dfs.append(tmp)
    for c in STD:
        tmp = train.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    # for c in SUM:
    #     tmp = train.groupby(['session_id','level_group'])[c].agg('sum')
    #     tmp.name = tmp.name + '_sum'
    #     dfs.append(tmp)
        
    for c in EVENTS:
        train[c] = (train.event_name == c).astype('int8')
    for c in EVENTS + ['elapsed_time']:
        tmp = train.groupby(['session_id','level_group'])[c].agg('sum')
        tmp.name = tmp.name + '_sum'
        dfs.append(tmp)
        
    train = train.drop(EVENTS,axis=1)
    df = pd.concat(dfs,axis=1)
    # df = df.fillna(-1)
    df = df.reset_index()
    df = df.set_index('session_id')
    return df

In [8]:
%%time
df = feature_engineer(train)
print( df.shape )
df.head()

(35337, 61)
CPU times: user 1min 43s, sys: 8.14 s, total: 1min 51s
Wall time: 1min 52s


,level_group,fqid_nunique,room_fqid_nunique,text_nunique,text_count,fqid_count,elapsed_time_max,hover_duration_max,index_max,level_max,...,navigate_click_sum,person_click_sum,cutscene_click_sum,object_click_sum,map_hover_sum,notification_click_sum,map_click_sum,observation_click_sum,checkpoint_sum,notebook_click_sum
session_id,,,,,,,,,,,,,,,,,,,,,
20090312431273200,0-4,30,7,56,62,104,194860,7899.0,164,4,...,81.0,22.0,28.0,11.0,4.0,8,2.0,4,1,0.0
20090312431273200,13-22,49,12,168,196,313,1272679,4750.0,931,22,...,170.0,123.0,60.0,20.0,14.0,10,6.0,3,1,0.0
20090312431273200,5-12,39,11,124,126,235,499235,4183.0,470,12,...,103.0,104.0,12.0,28.0,9.0,9,8.0,1,1,0.0
20090312433251036,0-4,22,6,49,61,97,233752,5567.0,138,4,...,49.0,18.0,36.0,15.0,3.0,5,3.0,2,1,2.0
20090312433251036,13-22,73,16,183,229,762,3815334,22351.0,1875,22,...,637.0,145.0,65.0,83.0,186.0,14,45.0,5,1,50.0


In [ ]:
df_target_agg_tn = df.groupby(['session_id'])[('text_nunique')].agg(['mean','max','min','std','sum'])
df_target_agg_tn.columns = df_target_agg_tn.columns+"_tn"
df_target_agg_tn_col  = df_target_agg_tn.columns.to_list()

df_target_agg_rn = df.groupby(['session_id'])[('room_fqid_nunique')].agg(['mean','max','min','std','sum'])
df_target_agg_rn.columns = df_target_agg_rn.columns+"_rn"
df_target_agg_rn_col  = df_target_agg_rn.columns.to_list()

df_target_agg_fn = df.groupby(['session_id'])[('fqid_nunique')].agg(['mean','max','min','std','sum'])
df_target_agg_fn.columns = df_target_agg_fn.columns+"_fn"
df_target_agg_fn_col  = df_target_agg_fn.columns.to_list()

df_target_agg_em = df.groupby(['session_id'])[('elapsed_time_mean')].agg(['mean','max','min','std','sum'])
df_target_agg_em.columns = df_target_agg_em.columns+"_em"
df_target_agg_em_col  = df_target_agg_em.columns.to_list()

df_target_agg_lm = df.groupby(['session_id'])[('level_mean')].agg(['mean','max','min','std','sum'])
df_target_agg_lm.columns = df_target_agg_lm.columns+"_lm"
df_target_agg_lm_col  = df_target_agg_lm.columns.to_list()

df_target_agg_pm = df.groupby(['session_id'])[('page_mean')].agg(['mean','max','min','std','sum'])
df_target_agg_pm.columns = df_target_agg_pm.columns+"_pm"
df_target_agg_pm_col  = df_target_agg_pm.columns.to_list()

df_target_agg_hm = df.groupby(['session_id'])[('hover_duration_mean')].agg(['mean','max','min','std','sum'])
df_target_agg_hm.columns = df_target_agg_hm.columns+"_hm"
df_target_agg_hm_col  = df_target_agg_hm.columns.to_list()

df_target_agg_im = df.groupby(['session_id'])[('index_mean')].agg(['mean','max','min','std','sum'])
df_target_agg_im.columns = df_target_agg_im.columns+"_im"
df_target_agg_im_col  = df_target_agg_im.columns.to_list()



df_target_agg_emx = df.groupby(['session_id'])[('elapsed_time_max')].agg(['mean','max','min','std','sum'])
df_target_agg_emx.columns = df_target_agg_emx.columns+"_emx"
df_target_agg_emx_col  = df_target_agg_emx.columns.to_list()

df_target_agg_emi = df.groupby(['session_id'])[('elapsed_time_min')].agg(['mean','max','min','std','sum'])
df_target_agg_emi.columns = df_target_agg_emi.columns+"_emi"
df_target_agg_emi_col  = df_target_agg_emi.columns.to_list()

df_target_agg_hmx = df.groupby(['session_id'])[('hover_duration_max')].agg(['mean','max','min','std','sum'])
df_target_agg_hmx.columns = df_target_agg_hmx.columns+"_hmx"
df_target_agg_hmx_col  = df_target_agg_hmx.columns.to_list()

df_target_agg_hmi = df.groupby(['session_id'])[('hover_duration_min')].agg(['mean','max','min','std','sum'])
df_target_agg_hmi.columns = df_target_agg_hmi.columns+"_hmi"
df_target_agg_hmi_col  = df_target_agg_hmi.columns.to_list()


df = pd.merge(df,df_target_agg_emx,left_index=True, right_index=True)
df = pd.merge(df,df_target_agg_emi,left_index=True, right_index=True)
df = pd.merge(df,df_target_agg_hmx,left_index=True, right_index=True)
df = pd.merge(df,df_target_agg_hmi,left_index=True, right_index=True)


df = pd.merge(df,df_target_agg_im,left_index=True, right_index=True)
df = pd.merge(df,df_target_agg_tn,left_index=True, right_index=True)
df = pd.merge(df,df_target_agg_rn,left_index=True, right_index=True)
df = pd.merge(df,df_target_agg_fn,left_index=True, right_index=True)
df = pd.merge(df,df_target_agg_em,left_index=True, right_index=True)
df = pd.merge(df,df_target_agg_lm,left_index=True, right_index=True)
df = pd.merge(df,df_target_agg_pm,left_index=True, right_index=True)
df = pd.merge(df,df_target_agg_hm,left_index=True, right_index=True)

In [9]:
FEATURES = [c for c in df.columns if c != 'level_group']
print('We will train with', len(FEATURES) ,'features')
ALL_USERS = df.index.unique()
print('We will train with', len(ALL_USERS) ,'users info')

We will train with 60 features
We will train with 11779 users info


In [ ]:
gkf = GroupKFold(n_splits=20)
oof = pd.DataFrame(data=np.zeros((len(ALL_USERS),18)), index=ALL_USERS)
models = {}

# COMPUTE CV SCORE WITH 5 GROUP K FOLD
for i, (train_index, test_index) in enumerate(gkf.split(X=df, groups=df.index)):
    print(' ')
    print('Fold',i+1)
    print(' ')
    
    xgb_params = {
    'objective' : 'binary:logistic',
    'eval_metric':'logloss',
    'learning_rate': 0.05,
    'max_depth': 4,
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'tree_method':'hist',
    'subsample':0.8,
    'colsample_bytree': 0.58,
    'use_label_encoder' : False}
    
    # ITERATE THRU QUESTIONS 1 THRU 18
    for t in range(1,19):
        
        # USE THIS TRAIN DATA WITH THESE QUESTIONS
        if t<=3: grp = '0-4'
        elif t<=13: grp = '5-12'
        elif t<=22: grp = '13-22'
            
        # TRAIN DATA
        train_x = df.iloc[train_index]
        train_x = train_x.loc[train_x.level_group == grp]
        train_users = train_x.index.values
        train_y = targets.loc[targets.q==t].set_index('session').loc[train_users]
        
        # VALID DATA
        valid_x = df.iloc[test_index]
        valid_x = valid_x.loc[valid_x.level_group == grp]
        valid_users = valid_x.index.values
        valid_y = targets.loc[targets.q==t].set_index('session').loc[valid_users]
        
        # TRAIN MODEL        
        clf =  XGBClassifier(**xgb_params)
        clf.fit(train_x[FEATURES].astype('float32'), train_y['correct'],
                eval_set=[ (valid_x[FEATURES].astype('float32'), valid_y['correct']) ],
                verbose=0)
        print(f'{t}({clf.best_ntree_limit}), ',end='')
        
        # SAVE MODEL, PREDICT VALID OOF
        models[f'{grp}_{t}'] = clf
        oof.loc[valid_users, t-1] = clf.predict_proba(valid_x[FEATURES].astype('float32'))[:,1]
        
    print()

 
Fold 1
 
1(1491), 2(1204), 3(763), 4(1040), 5(912), 6(626), 7(604), 8(454), 9(1312), 10(869), 11(955), 12(619), 13(639), 14(806), 15(1005), 16(700), 17(654), 18(1331), 
 
Fold 2
 
1(1454), 2(849), 3(1115), 4(1242), 5(655), 6(770), 7(1338), 8(573), 9(720), 10(845), 11(877), 12(653), 13(629), 14(647), 15(962), 16(438), 17(369), 18(864), 
 
Fold 3
 
1(1076), 2(825), 3(821), 4(827), 5(562), 6(525), 7(1266), 8(395), 9(712), 10(611), 11(815), 12(939), 13(873), 14(740), 15(681), 16(1160), 17(646), 18(1299), 
 
Fold 4
 
1(1263), 2(1050), 3(1061), 4(1055), 5(1118), 6(954), 7(659), 8(445), 9(394), 10(947), 11(1198), 12(637), 13(575), 14(1069), 15(496), 16(449), 17(663), 18(897), 
 
Fold 5
 
1(1410), 2(891), 3(1158), 4(1425), 5(554), 6(1279), 7(795), 8(413), 9(1497), 10(628), 11(549), 12(1008), 13(805), 14(639), 15(1338), 16(936), 17(620), 18(746), 
 
Fold 6
 
1(1384), 2(869), 3(1074), 4(1488), 5(778), 6(694), 7(819), 8(654), 9(958), 10(839), 11(702), 12(943), 13(795), 14(1000), 15(853), 16(640

In [ ]:
true = oof.copy()
for k in range(18):
    # GET TRUE LABELS
    tmp = targets.loc[targets.q == k+1].set_index('session').loc[ALL_USERS]
    true[k] = tmp.correct.values

In [ ]:
scores = []; thresholds = []
best_score = 0; best_threshold = 0

for threshold in np.arange(0.4,0.81,0.01):
    print(f'{threshold:.02f}, ',end='')
    preds = (oof.values.reshape((-1))>threshold).astype('int')
    m = f1_score(true.values.reshape((-1)), preds, average='macro')   
    scores.append(m)
    thresholds.append(threshold)
    if m>best_score:
        best_score = m
        best_threshold = threshold

In [ ]:
print('When using optimal threshold...')
for k in range(18):
        
    # COMPUTE F1 SCORE PER QUESTION
    m = f1_score(true[k].values, (oof[k].values>best_threshold).astype('int'), average='macro')
    print(f'Q{k}: F1 =',m)
    
# COMPUTE F1 SCORE OVERALL
m = f1_score(true.values.reshape((-1)), (oof.values.reshape((-1))>best_threshold).astype('int'), average='macro')
print('==> Overall F1 =',m)

In [ ]:
# IMPORT KAGGLE API
import jo_wilder
env = jo_wilder.make_env()
iter_test = env.iter_test()

# CLEAR MEMORY
import gc
del train, targets, df, oof, true
_ = gc.collect()

In [ ]:
limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}

for (sample_submission, test) in iter_test:
    
    test['room_coor_x'] = test['room_coor_x'].fillna(0)
    test['room_coor_y'] = test['room_coor_y'].fillna(0)
    test['screen_coor_x'] = test['screen_coor_x'].fillna(0)
    test['screen_coor_y'] = test['screen_coor_y'].fillna(0)
    test['hover_duration'] = test['hover_duration'].fillna(0)
    test['page'] = test['page'].fillna(-1)
    # FEATURE ENGINEER TEST DATA
    df = feature_engineer(test)


    df_target_agg_tn = df.groupby(['session_id'])[('text_nunique')].agg(['mean','max','min','std','sum'])
    df_target_agg_tn.columns = df_target_agg_tn.columns+"_tn"
    df_target_agg_tn_col  = df_target_agg_tn.columns.to_list()

    df_target_agg_rn = df.groupby(['session_id'])[('room_fqid_nunique')].agg(['mean','max','min','std','sum'])
    df_target_agg_rn.columns = df_target_agg_rn.columns+"_rn"
    df_target_agg_rn_col  = df_target_agg_rn.columns.to_list()

    df_target_agg_fn = df.groupby(['session_id'])[('fqid_nunique')].agg(['mean','max','min','std','sum'])
    df_target_agg_fn.columns = df_target_agg_fn.columns+"_fn"
    df_target_agg_fn_col  = df_target_agg_fn.columns.to_list()

    df_target_agg_em = df.groupby(['session_id'])[('elapsed_time_mean')].agg(['mean','max','min','std','sum'])
    df_target_agg_em.columns = df_target_agg_em.columns+"_em"
    df_target_agg_em_col  = df_target_agg_em.columns.to_list()

    df_target_agg_lm = df.groupby(['session_id'])[('level_mean')].agg(['mean','max','min','std','sum'])
    df_target_agg_lm.columns = df_target_agg_lm.columns+"_lm"
    df_target_agg_lm_col  = df_target_agg_lm.columns.to_list()

    df_target_agg_pm = df.groupby(['session_id'])[('page_mean')].agg(['mean','max','min','std','sum'])
    df_target_agg_pm.columns = df_target_agg_pm.columns+"_pm"
    df_target_agg_pm_col  = df_target_agg_pm.columns.to_list()

    df_target_agg_hm = df.groupby(['session_id'])[('hover_duration_mean')].agg(['mean','max','min','std','sum'])
    df_target_agg_hm.columns = df_target_agg_hm.columns+"_hm"
    df_target_agg_hm_col  = df_target_agg_hm.columns.to_list()

    df_target_agg_im = df.groupby(['session_id'])[('index_mean')].agg(['mean','max','min','std','sum'])
    df_target_agg_im.columns = df_target_agg_im.columns+"_im"
    df_target_agg_im_col  = df_target_agg_im.columns.to_list()



    df_target_agg_emx = df.groupby(['session_id'])[('elapsed_time_max')].agg(['mean','max','min','std','sum'])
    df_target_agg_emx.columns = df_target_agg_emx.columns+"_emx"
    df_target_agg_emx_col  = df_target_agg_emx.columns.to_list()

    df_target_agg_emi = df.groupby(['session_id'])[('elapsed_time_min')].agg(['mean','max','min','std','sum'])
    df_target_agg_emi.columns = df_target_agg_emi.columns+"_emi"
    df_target_agg_emi_col  = df_target_agg_emi.columns.to_list()

    df_target_agg_hmx = df.groupby(['session_id'])[('hover_duration_max')].agg(['mean','max','min','std','sum'])
    df_target_agg_hmx.columns = df_target_agg_hmx.columns+"_hmx"
    df_target_agg_hmx_col  = df_target_agg_hmx.columns.to_list()

    df_target_agg_hmi = df.groupby(['session_id'])[('hover_duration_min')].agg(['mean','max','min','std','sum'])
    df_target_agg_hmi.columns = df_target_agg_hmi.columns+"_hmi"
    df_target_agg_hmi_col  = df_target_agg_hmi.columns.to_list()


    df = pd.merge(df,df_target_agg_emx,left_index=True, right_index=True)
    df = pd.merge(df,df_target_agg_emi,left_index=True, right_index=True)
    df = pd.merge(df,df_target_agg_hmx,left_index=True, right_index=True)
    df = pd.merge(df,df_target_agg_hmi,left_index=True, right_index=True)


    df = pd.merge(df,df_target_agg_im,left_index=True, right_index=True)
    df = pd.merge(df,df_target_agg_tn,left_index=True, right_index=True)
    df = pd.merge(df,df_target_agg_rn,left_index=True, right_index=True)
    df = pd.merge(df,df_target_agg_fn,left_index=True, right_index=True)
    df = pd.merge(df,df_target_agg_em,left_index=True, right_index=True)
    df = pd.merge(df,df_target_agg_lm,left_index=True, right_index=True)
    df = pd.merge(df,df_target_agg_pm,left_index=True, right_index=True)
    df = pd.merge(df,df_target_agg_hm,left_index=True, right_index=True)

    # INFER TEST DATA
    grp = test.level_group.values[0]
    a,b = limits[grp]
    for t in range(a,b):
        clf = models[f'{grp}_{t}']
        p = clf.predict_proba(df[FEATURES].astype('float32'))[:,1]
        mask = sample_submission.session_id.str.contains(f'q{t}')
        sample_submission.loc[mask,'correct'] = int(p.item()>best_threshold)
    
    env.predict(sample_submission)

In [ ]:
df = pd.read_csv('submission.csv')
print( df.shape )
df.head()

In [ ]:
print(df.correct.mean())